## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
#載入套件
from keras.models import Sequential  #用來啟動 NN
from keras.layers import Convolution2D # Convolution Operation
from keras.layers import MaxPooling2D # Pooling
from keras.layers import Flatten
from keras.layers import Dense # Fully Connected Networks
from keras.layers import Dropout 
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
# CUDNN_STATUS_INTERNAL_ERROR 於運行時出現, 重裝 cudnn 之後無效, suspect gpu ram issue
# 利用動態分配GPU內存 problem solved.
# ref: https://github.com/keras-team/keras/issues/4161
def dynam_GPU_ram():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
    config.log_device_placement = True  # to log device placement (on which device the operation ran)
    sess = tf.compat.v1.Session(config=config)
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

    
    
dynam_GPU_ram()

config = tf.ConfigProto(allow_soft_placement=True) 
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data() # 載入 Cifar-10 資料集
"""
x_train, x_test : uint8數組表示的RGB圖像數據，尺寸為(num_samples, 3, 32, 32)或(num_samples, 32, 32, 3)
y_train, y_test : uint8數組表示的類別標籤（範圍在0~9之間的整數），尺寸為(num_samples,)。
"""
print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
"""
常見的前處理包含Normalization 和 Standardization，
狹義的Normalization 指的是將所有資料rescale到0-1之間，
在影像中/255就是其中一種，上方程式部分比較正確的名稱是Standardization，
也就是將所有的資料-平均值/STD，之所以會加上一個很小的值是為了避免分母為0，
算是廣義的Normalization ，這種方式結果有正有負。
"""
def normalize(X_train,X_test):
        # 計算平均值
        mean = np.mean(X_train,axis=(0,1,2,3)) # np.mean(axis=(0,1,2,3))輸出的矩陣維度為4維。
        # 計算標準差
        std = np.std(X_train, axis=(0, 1, 2, 3))# np.std(axis=(0,1,2,3))輸出的矩陣維度為4維。
        # 透過平均值與標準差將X_train的資料歸一化
        X_train = (X_train-mean)/(std+1e-7)# 加上一個很小的值(1e-7)，為了避免分母為0。
        # 透過平均值與標準差將X_test的資料歸一化
        X_test = (X_test-mean)/(std+1e-7)# 加上一個很小的值(1e-7)，為了避免分母為0。
        
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [5]:
# 建立簡單的線性執行的模型
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters= 32,kernel_size=(3,3),input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2))) # 建立池化層，池化大小=2x2，取最大值

#卷積組合
classifier.add(Convolution2D(filters= 32,kernel_size=(3,3),activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax')) # 使用 softmax activation function，並將結果分類
"""
Softmax函數能將輸出值總合統整成1，
轉換成機率的型態，通常用於多類的分類器輸出。
"""
# 超過兩個就要選categorical_crossentrophy
# 編譯: 選擇損失函數、優化方法及成效衡量方式
# 確立目標及求解方法：以compile函數定義損失函數(loss)、優化函數(optimizer)及成效衡量指標(mertrics)。
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 14s 279us/step - loss: 1.7222 - acc: 0.4912
Epoch 2/100
50000/50000 [==============================] - 11s 220us/step - loss: 0.9265 - acc: 0.6742
Epoch 3/100
50000/50000 [==============================] - 11s 217us/step - loss: 0.6666 - acc: 0.7691
Epoch 4/100
50000/50000 [==============================] - 10s 208us/step - loss: 0.4568 - acc: 0.8417
Epoch 5/100
50000/50000 [==============================] - 10s 204us/step - loss: 0.3006 - acc: 0.8956
Epoch 6/100
50000/50000 [==============================] - 10s 204us/step - loss: 0.2046 - acc: 0.9303
Epoch 7/100
50000/50000 [==============================] - 10s 204us/step - loss: 0.1655 - acc: 0.9435
Epoch 8/100
50000/50000 [==============================] - 10s 203us/step - loss: 0.1366 - acc: 0.9534
Epoch 9/100
50000/50000 [==============================] - 10s 204us/step - loss: 0.1238 - acc: 0.9577
Epoch 10/100
50000/50000 [==============================] - 10s 204us/ste

50000/50000 [==============================] - 11s 211us/step - loss: 0.0128 - acc: 0.9961
Epoch 80/100
50000/50000 [==============================] - 11s 227us/step - loss: 0.0144 - acc: 0.9964
Epoch 81/100
50000/50000 [==============================] - 12s 230us/step - loss: 0.0147 - acc: 0.9956
Epoch 82/100
50000/50000 [==============================] - 11s 222us/step - loss: 0.0114 - acc: 0.9966
Epoch 83/100
50000/50000 [==============================] - 11s 212us/step - loss: 0.0103 - acc: 0.9968
Epoch 84/100
50000/50000 [==============================] - 11s 216us/step - loss: 0.0143 - acc: 0.9958
Epoch 85/100
50000/50000 [==============================] - 11s 210us/step - loss: 0.0186 - acc: 0.9948
Epoch 86/100
50000/50000 [==============================] - 11s 218us/step - loss: 0.0149 - acc: 0.9958
Epoch 87/100
50000/50000 [==============================] - 11s 222us/step - loss: 0.0116 - acc: 0.9965
Epoch 88/100
50000/50000 [==============================] - 10s 209us/step - 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7)  # np.zeros 可以用來建立以 0 為元素的矩陣
classifier.predict(input_example)

array([[1.7962406e-05, 2.9658397e-08, 2.1406795e-05, 9.9895751e-01,
        1.0020374e-04, 1.8004990e-05, 6.2862737e-06, 8.3838170e-04,
        1.3337419e-05, 2.6792508e-05]], dtype=float32)